# 회원 탈퇴 예측
- 클러스터링 행동 분석은 사용방법에 따라 많은 가능성이 있는 기술이다.
- 행동패턴을 분석 할 수 있으면 어떤 고객이 탈퇴할지와 같은 예측도 가능.
- 탈퇴 방지를 하기 위해 정책을 준비 하는 것도 가능.

In [59]:
import warnings
warnings.filterwarnings('ignore')

In [60]:
import pandas as pd

In [61]:
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

In [62]:
year_months = list(uselog_months['연월'].unique())
year_months[0]

201804

In [63]:
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months['연월'] == year_months[i]]
    tmp.rename(columns={'count':'count_0'}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months['연월'] == year_months[i-1]]
    del tmp_before['연월']
    tmp_before.rename(columns={'count':'count_1'}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on = 'customer_id', how = 'left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)

In [64]:
uselog.head()

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### 탈퇴 전월의 탈퇴 고객 데이터를 작성
- 탈퇴한 월이 아닌 탈퇴 전월의 데터를 작성 한다.
- 탈퇴를 예측하는 목적은 탈퇴 방지를 위한 것.
- 해당 월에 탈퇴 신성 하고 다음달 말일에 탈퇴 처리 함.
- 탈퇴 전월로부터 탈퇴 신성 확률을 예측

In [65]:
from dateutil.relativedelta import relativedelta

In [66]:
exit_customer = customer.loc[customer['is_deleted'] == 1]
exit_customer['exit_date'] = None
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])

In [67]:
for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(months=1)

In [68]:
exit_customer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1350 entries, 708 to 4099
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   customer_id        1350 non-null   object        
 1   name               1350 non-null   object        
 2   class              1350 non-null   object        
 3   gender             1350 non-null   object        
 4   start_date         1350 non-null   object        
 5   end_date           1350 non-null   datetime64[ns]
 6   campaign_id        1350 non-null   object        
 7   is_deleted         1350 non-null   int64         
 8   class_name         1350 non-null   object        
 9   price              1350 non-null   int64         
 10  campaign_name      1350 non-null   object        
 11  mean               1350 non-null   float64       
 12  median             1350 non-null   float64       
 13  max                1350 non-null   int64         
 14  min   

In [69]:
exit_customer['exit_date'] = pd.to_datetime(exit_customer['exit_date'])
exit_customer['연월'] = exit_customer['exit_date'].dt.strftime('%Y%m')
exit_customer.head(3)

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,연월
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,0_종일,10500,2_일반,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30,201803
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,1_주간,7500,2_일반,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30,201803
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,0_종일,10500,2_일반,1.0,1.0,1,1,0,2018-04-30,23,2018-03-30,201803


In [70]:
uselog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33851 entries, 0 to 33850
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   연월           33851 non-null  int64  
 1   customer_id  33851 non-null  object 
 2   count_0      33851 non-null  int64  
 3   count_1      32650 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.0+ MB


In [71]:
uselog['연월'] = uselog['연월'].astype(str)

In [72]:
exit_uselog = pd.merge(uselog, exit_customer, on = ['customer_id', '연월'], how = 'left')

In [73]:
exit_uselog.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [74]:
len(exit_uselog)

33851

In [75]:
exit_uselog = exit_uselog.dropna(subset=['name'])
print(len(exit_uselog))
print(len(exit_uselog['customer_id'].unique()))
exit_uselog.head()

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,2_일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,2_일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,2_일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


### 지속회원의 데이터 작성
- 탈퇴 회원의 데이터 수는 1104, 지속회원 데이터 수는 name의 결측치를 제거 했읍에도 27422개, 데이터 불균형이 있음.
- 샘플 수를 조정, 모든 기간의 회원 데이터를 사용하지 않고, 하나의 기간만 사용.

In [76]:
conti_customer = customer.loc[customer['is_deleted'] == 0]
conti_uselog = pd.merge(uselog, conti_customer, on = ['customer_id'], how = 'left')

In [77]:
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=['name'])
print(len(conti_uselog))

33851
27422


In [78]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset='customer_id')
print(len(conti_uselog))
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201806,GD040768,5,7.0,XXXX,C02,M,2016-09-01,NaN,CA1,...,1_주간,7500.0,2_일반,4.666667,5.0,7.0,1.0,1.0,2019-04-30,31.0
1,201903,AS707671,5,5.0,XXXX,C02,F,2017-06-01,NaN,CA1,...,1_주간,7500.0,2_일반,6.000000,5.0,10.0,4.0,1.0,2019-04-30,22.0
2,201806,GD081993,7,8.0,XXXXXX,C01,F,2016-08-01,NaN,CA1,...,0_종일,10500.0,2_일반,5.500000,5.5,9.0,2.0,1.0,2019-04-30,32.0
3,201809,PL764756,7,6.0,XXXXX,C03,M,2017-05-01,NaN,CA1,...,2_야간,6000.0,2_일반,5.916667,6.0,10.0,3.0,1.0,2019-04-30,23.0
4,201810,PL636860,5,5.0,XXXXX,C01,F,2017-05-01,NaN,CA1,...,0_종일,10500.0,2_일반,5.416667,5.0,8.0,4.0,1.0,2019-04-30,23.0


In [79]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201806,GD040768,5,7.0,XXXX,C02,M,2016-09-01,NaN,CA1,...,7500.0,2_일반,4.666667,5.0,7.0,1.0,1.0,2019-04-30,31.0,NaT
1,201903,AS707671,5,5.0,XXXX,C02,F,2017-06-01,NaN,CA1,...,7500.0,2_일반,6.000000,5.0,10.0,4.0,1.0,2019-04-30,22.0,NaT
2,201806,GD081993,7,8.0,XXXXXX,C01,F,2016-08-01,NaN,CA1,...,10500.0,2_일반,5.500000,5.5,9.0,2.0,1.0,2019-04-30,32.0,NaT
3,201809,PL764756,7,6.0,XXXXX,C03,M,2017-05-01,NaN,CA1,...,6000.0,2_일반,5.916667,6.0,10.0,3.0,1.0,2019-04-30,23.0,NaT
4,201810,PL636860,5,5.0,XXXXX,C01,F,2017-05-01,NaN,CA1,...,10500.0,2_일반,5.416667,5.0,8.0,4.0,1.0,2019-04-30,23.0,NaT


### 예측할 달의 재적기간 작성

In [80]:
predict_data['period'] = 0
predict_data['now_date'] = pd.to_datetime(predict_data['연월'], format='%Y%m')
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data['now_date'][i], predict_data['start_date'][i])
    predict_data['period'][i] = int(delta.years*12 + delta.months)

In [81]:
predict_data.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201806,GD040768,5,7.0,XXXX,C02,M,2016-09-01,NaN,CA1,...,4.666667,5.0,7.0,1.0,1.0,2019-04-30,31.0,NaT,21,2018-06-01
1,201903,AS707671,5,5.0,XXXX,C02,F,2017-06-01,NaN,CA1,...,6.000000,5.0,10.0,4.0,1.0,2019-04-30,22.0,NaT,21,2019-03-01
2,201806,GD081993,7,8.0,XXXXXX,C01,F,2016-08-01,NaN,CA1,...,5.500000,5.5,9.0,2.0,1.0,2019-04-30,32.0,NaT,22,2018-06-01
3,201809,PL764756,7,6.0,XXXXX,C03,M,2017-05-01,NaN,CA1,...,5.916667,6.0,10.0,3.0,1.0,2019-04-30,23.0,NaT,16,2018-09-01
4,201810,PL636860,5,5.0,XXXXX,C01,F,2017-05-01,NaN,CA1,...,5.416667,5.0,8.0,4.0,1.0,2019-04-30,23.0,NaT,17,2018-10-01


In [82]:
predict_data.isna().sum() / len(predict_data)

연월                   0.000000
customer_id          0.000000
count_0              0.000000
count_1              0.063862
name                 0.000000
class                0.000000
gender               0.000000
start_date           0.000000
end_date             0.720223
campaign_id          0.000000
is_deleted           0.000000
class_name           0.000000
price                0.000000
campaign_name        0.000000
mean                 0.000000
median               0.000000
max                  0.000000
min                  0.000000
routine_flg          0.000000
calc_date            0.000000
membership_period    0.000000
exit_date            0.720223
period               0.000000
now_date             0.000000
dtype: float64

In [83]:
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.isna().sum() / len(predict_data)

연월                   0.000000
customer_id          0.000000
count_0              0.000000
count_1              0.000000
name                 0.000000
class                0.000000
gender               0.000000
start_date           0.000000
end_date             0.715214
campaign_id          0.000000
is_deleted           0.000000
class_name           0.000000
price                0.000000
campaign_name        0.000000
mean                 0.000000
median               0.000000
max                  0.000000
min                  0.000000
routine_flg          0.000000
calc_date            0.000000
membership_period    0.000000
exit_date            0.715214
period               0.000000
now_date             0.000000
dtype: float64

### 카테고리 변수 처리 및 가공

In [84]:
predict_data.columns

Index(['연월', 'customer_id', 'count_0', 'count_1', 'name', 'class', 'gender',
       'start_date', 'end_date', 'campaign_id', 'is_deleted', 'class_name',
       'price', 'campaign_name', 'mean', 'median', 'max', 'min', 'routine_flg',
       'calc_date', 'membership_period', 'exit_date', 'period', 'now_date'],
      dtype='object')

In [85]:
target_col = ['campaign_name', 'class_name', 'gender', 'count_1', 'routine_flg', 'period', 'is_deleted']
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,2_일반,1_주간,M,7.0,1.0,21,0.0
1,2_일반,1_주간,F,5.0,1.0,21,0.0
2,2_일반,0_종일,F,8.0,1.0,22,0.0
3,2_일반,2_야간,M,6.0,1.0,16,0.0
4,2_일반,0_종일,F,5.0,1.0,17,0.0


In [86]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,campaign_name_2_일반,class_name_0_종일,class_name_1_주간,class_name_2_야간,gender_F,gender_M
0,7.0,1.0,21,0.0,0,0,1,0,1,0,0,1
1,5.0,1.0,21,0.0,0,0,1,0,1,0,1,0
2,8.0,1.0,22,0.0,0,0,1,1,0,0,1,0
3,6.0,1.0,16,0.0,0,0,1,0,0,1,0,1
4,5.0,1.0,17,0.0,0,0,1,1,0,0,1,0


In [87]:
del predict_data['campaign_name_2_일반']
del predict_data['class_name_2_야간']
del predict_data['gender_M']
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_0_입회비반액할인,campaign_name_1_입회비무료,class_name_0_종일,class_name_1_주간,gender_F
0,7.0,1.0,21,0.0,0,0,0,1,0
1,5.0,1.0,21,0.0,0,0,0,1,1
2,8.0,1.0,22,0.0,0,0,1,0,1
3,6.0,1.0,16,0.0,0,0,0,0,0
4,5.0,1.0,17,0.0,0,0,1,0,1


### 의사결정나무 알고리즘을 사용 해서 탈퇴 예측 모델 

In [88]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [89]:
exit = predict_data.loc[predict_data['is_deleted'] == 1]
conti = predict_data.loc[predict_data['is_deleted'] == 0].sample(len(exit))

In [90]:
X['period'].unique()

array([7], dtype=int64)

In [91]:
X = pd.concat([exit, conti], ignore_index=True)
y = X['is_deleted']

In [92]:
del X['is_deleted']

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [94]:
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [95]:
pred

array([0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1.,
       0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0.,
       1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
       1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0.,
       1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0.,
       0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 0.

In [96]:
results_test = pd.DataFrame({'y_test': y_test, 'y_pred': pred})
results_test.head()

,y_test,y_pred
1091,0.0,0.0
1786,0.0,0.0
1439,0.0,1.0
745,1.0,1.0
820,1.0,0.0


### 예측 모델 평가

In [97]:
correct = len(results_test.loc[results_test['y_test'] == results_test['y_pred']])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.9049881235154394


In [98]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9049881235154394
0.978015448603684


In [99]:
model = DecisionTreeClassifier(max_depth=5, random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9121140142517815
0.93048128342246


# 모델에 기여하고 있는 변수 확인
- 1개월 전 이용 횟수, 정기 이용 여부 기여 하고 있는 것을 확인

In [100]:
importance = pd.DataFrame({'feature_name': X.columns, 'feature_importances': model.feature_importances_})
importance

,feature_name,feature_importances
0,count_1,0.363610
1,routine_flg,0.128899
2,period,0.505152
3,campaign_name_0_입회비반액할인,0.002015
4,campaign_name_1_입회비무료,0.000315
5,class_name_0_종일,0.000000
6,class_name_1_주간,0.000000
7,gender_F,0.000010


### 회원의 탈퇴 예측

In [101]:
count_1 = 3
routing_flg = 1
period = 10
campaign_name = "입회비무료"
class_name = "종일"
gender = "M"

In [103]:
if campaign_name == "입회비반값할인":
    campaign_name_list = [1, 0]
elif campaign_name == '입회비무료':
    campaign_name_list = [0, 1]
elif campaign_name == '일반':
    campaign_name_list = [0, 0]
if class_name == "종일":
    class_name_list = [1, 0]
elif class_name == "주간":
    class_name_list = [0, 1]
elif class_name == "야간":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]

input_data = [count_1, routing_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [104]:
input_data

[3, 1, 10, 0, 1, 1, 0, 0]

In [105]:
print(model.predict([input_data]))
# AUC-ROC계산을 위해 각 클래스에 확률이 필요.
print(model.predict_proba([input_data]))

[1.]
[[0. 1.]]
